In [1]:
# -*- coding:utf-8 -*-

import torch
import json
from glm1.modeling_chatglm import ChatGLMForConditionalGeneration
from glm1.tokenization_chatglm import ChatGLMTokenizer
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from tqdm import tqdm
import time
import os
import argparse


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /root/miniconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Shanghai')}
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.31.1.127'), PosixPath('http'), PosixPath('5

In [2]:
model = ChatGLMForConditionalGeneration.from_pretrained("/root/autodl-tmp/ptms/THUDM-chatglm-6b/")
tokenizer = ChatGLMTokenizer.from_pretrained("/root/autodl-tmp/KB_LLM/sft/ChatGLM-Finetuning/output-glm-lora/epoch-10-step-4690")
model.eval()
model = PeftModel.from_pretrained(model, "/root/autodl-tmp/KB_LLM/sft/ChatGLM-Finetuning/output-glm-lora/epoch-10-step-4690", torch_dtype=torch.float32)
model.half().to("cuda:{}".format(0))
model.eval()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (word_embeddings): Embedding(130528, 4096)
        (layers): ModuleList(
          (0-27): 28 x GLMBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): SelfAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=12288, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embed

# dev

In [33]:
from tqdm import tqdm_notebook
max_len=1560
max_src_len = 1124
max_tgt_len = max_len - max_src_len - 3
dev_path="/root/autodl-tmp/KB_LLM/data/dev_sft.json"
# generate = model.generate()
sums_table, count_table = 0.0, 0.0
true_table,false_table=[],[]

sums_tuple, count_tuple = 0.0, 0.0
true_tuple,false_tuple=[],[]

with torch.no_grad():
    
    max_tgt_len = max_len - max_src_len - 3
    with open(dev_path, "r", encoding="utf-8") as fh:
        data = fh.readlines()
        for i, line in tqdm_notebook(enumerate(data)):
            # if i<100:
            sample = json.loads(line.strip())
            # if sample["task_type"] == "table_extract":
            #     prompt = "已知下面表格信息：" + "{}，\n问：{}\n答：".format(sample["instruction"], sample["question"])
            #     response, history=model.chat(tokenizer,prompt,max_length=2048)
            #     sums_table+=1
            #     # if response=="唐":
            #     #     break
            #     #     exit()
            #     # print("***response***")
            #     # print(response)
            #     # print("***answer***")
            #     # print(sample["answer"])
            #     if sample["answer"] == response or response.strip()=="":
            #         count_table += 1
            #         true_table.append((response,sample["answer"]))
            #     else:
            #         false_table.append((response,sample["answer"]))
            if sample["task_type"] == "tuple_extract":
                prompt = "请抽取下面问句的主宾二元组，格式xx|xx。问句："+ sample["question"]
                # prompt=f"""问题:'{sample["question"]}'，请从已知内容中抽取答案。如果无法从中得到答案，请说 "没有找到该问题对应的知识"。已知内容:{sample["instruction"]}"""
                response, history=model.chat(tokenizer,prompt,max_length=2048)
                sums_tuple+=1
                if sample["answer"] == ";".join(response.replace("|||","").split("；")):
                    count_tuple += 1
                    true_tuple.append((response,sample["answer"]))
                else:
                    false_tuple.append((response,sample["answer"]))

/tmp/ipykernel_5727/2206282613.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, line in tqdm_notebook(enumerate(data)):


0it [00:00, ?it/s]

In [10]:
count_table/sums_table,

(0.7450980392156863, 0.0)

In [34]:
count_tuple/sums_tuple

0.774390243902439

In [32]:
sample["answer"],";".join(response.replace("|||","").split("；"))

('三上枝织别名;三上枝织中文名;三上枝织外文名;三上枝织国籍;三上枝织民族;三上枝织出生年月;三上枝织职业;三上枝织毕业院校;三上枝织经纪公司;三上枝织代表作品;三上枝织出道时间;三上枝织所属团体',
 '葛珊珊别名;葛珊珊中文名;葛珊珊国籍;葛珊珊民族;葛珊珊星座;葛珊珊血型;葛珊珊出生地;葛珊珊出生日期;葛珊珊职业;葛珊珊经纪公司;葛珊珊生肖;葛珊珊丈夫;葛珊珊师傅;葛珊珊赵本山徒弟;葛珊珊签约公司;葛珊珊籍贯;葛珊珊家庭成员')

In [22]:
sample = json.loads(data[2].strip())
prompt = "请抽取下面问句的主宾二元组，格式xx|xx。问句："+ sample["question"]
response, history = model.chat(tokenizer,prompt,max_length=max_tgt_len)

In [26]:
";".join(response.replace("|||","").split("；"))==sample["answer"]

True

In [24]:
sample["answer"]

'葛珊珊别名;葛珊珊中文名;葛珊珊国籍;葛珊珊民族;葛珊珊星座;葛珊珊血型;葛珊珊出生地;葛珊珊出生日期;葛珊珊职业;葛珊珊经纪公司;葛珊珊生肖;葛珊珊丈夫;葛珊珊师傅;葛珊珊赵本山徒弟;葛珊珊签约公司;葛珊珊籍贯;葛珊珊家庭成员'

# test

## 抽取 实体-关系词二元组

In [35]:

import json
data=json.loads(open("../../result/es_top20_test.json",encoding="utf8").read())

In [8]:
data[-14]

{'id': 'TE000875',
 'question': '汪小敏喜欢哪一部电视剧？',
 'attribute': [['3561694', '汪小敏 ||| 喜欢的电视 ||| 《吸血鬼日记》'],
  ['3561697', '汪小敏 ||| 喜欢的动物 ||| 猫'],
  ['3561689', '汪小敏 ||| 喜欢的卡通 ||| Hello Kitty'],
  ['3561692', '汪小敏 ||| 喜欢的水果 ||| 橙、草莓'],
  ['41234838', '喜欢·一个人 ||| 类型 ||| 华人电视剧'],
  ['2033625', '张檬 ||| 喜欢电视剧类 ||| 国产电视剧、美剧'],
  ['3561693', '汪小敏 ||| 喜欢的游戏 ||| 模拟人生'],
  ['3561699', '汪小敏 ||| 喜欢的食物 ||| 甜品、面包'],
  ['6654877', '石述思 ||| 喜欢的电视剧 ||| 美剧'],
  ['1970425', '陈敏 ||| 喜欢 ||| 上网 看电视 编辑'],
  ['3561700', '汪小敏 ||| 喜欢的球队 ||| 洛杉矶湖人'],
  ['1422734', '韩一菲 ||| 喜欢的电视剧 ||| 《常回家看看》'],
  ['1312805', '陈亚兰 ||| 电视剧 ||| 欢喜菩萨'],
  ['370198', '六哲 ||| 最喜欢电视剧 ||| 《越狱》'],
  ['3561701', '汪小敏 ||| 喜欢的音乐 ||| 爵士、蓝调、乡村'],
  ['2403995', '欢喜婆婆俏媳妇 ||| 电视剧海报 ||| 电视剧海报'],
  ['39957188', '《皆大欢喜》(2001年香港电视剧) ||| 别名 ||| 《皆大欢喜》'],
  ['15512767', '欢喜(电视剧《师父·明白了》中角色) ||| 别名 ||| 欢喜'],
  ['39736456', '谢苗 ||| 最喜欢的电视 ||| 武侠剧'],
  ['40537522', '曾黎 ||| 喜欢的电视剧 ||| 《大宅门》']]}

In [39]:
from tqdm import tqdm
max_len=1560
max_src_len = 1124
max_tgt_len = max_len - max_src_len - 3
# generate = model.generate()
sums,count=0,0
with torch.no_grad():
    sums, count = 0.0, 0.0
    max_tgt_len = max_len - max_src_len - 3
    
    for i, sample in tqdm_notebook(enumerate(data[:-13])):
        # prompt = "已知下面表格信息：" + "{}，\n问：{}\n答：".format(";".join([k[1] for k in sample["attribute"]]), sample["question"])
        prompt = f"请抽取下面问句的主宾二元组，格式xx|xx。问句：{sample['question']}"
        response, history=model.chat(tokenizer,prompt,max_length=2048)
        print("***response***")
        print(sample["question"],response)
        sample["attribute"]=response.split("；")


/tmp/ipykernel_5727/485679519.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, sample in tqdm_notebook(enumerate(data[:-13])):


0it [00:00, ?it/s]

***response***
恩金·巴伊塔尔作为土耳其运动员，在球队中担负的职责是什么 恩金·巴伊塔尔|||所属运动队
***response***
迪迦·奥特曼的结束曲是哪一首？ 迪迦·奥特曼|||结尾曲
***response***
哪些官职在松平赖桓的身上 松平赖桓|||官职
***response***
什么是杰基·乔伊纳一克西获得的单人最好成就？ 杰基·乔伊纳一克西|||单人最佳成就
***response***
中国男子篮球职业联赛主要是什么运动？ 中国男子篮球职业联赛|||主要运动
***response***
我深致对周嘉诚的认同和敬佩，他的《中国高新企业指南》和《慌乱中的高新技术企业》深深的吸引了我。然而在各大论坛和网站上，我无法查询到他的主要成就究竟是什么，你能告诉我吗？ 周嘉诚|||主要成就
***response***
我知道王小诗是位非常出色的足球运动员，并且她在哪些位置上使用过？她的身高和体重分别是多少？她曾经在哪个青年队效力过，能够告诉我吗？ 王小诗|||身高；王小诗|||体重；王小诗|||所属青年队
***response***
龚文涛属于什么政治团体 龚文涛|||政治团体
***response***
深圳号导弹驱逐舰的英文名称是什么？ 深圳号|||英文别名
***response***
占城时期的冯兴能和哪些人对比呢 冯兴能|||对比
***response***
Amy Klinprathum是一位美国的演员，她的生日和身高都是我知道的，但我对她的一些其他信息知道的并不多。比如，她的外文名是什么，她的别名或者是昵称是什么，她的出生地在哪里，她的星座是什么，她的配偶是谁等等。如果方便的话，希望你能提供一下这方面的信息。 Amy Klinprathum|||外文名；Amy Klinprathum|||别名；Amy Klinprathum|||昵称；Amy Klinprathum|||出生地；Amy Klinprathum|||星座；Amy Klinprathum|||配偶
***response***
漫画706提到的巴托洛米奥的悬赏数额是多少贝利 巴托洛米奥|||赏金
***response***
高胜美惧怕的爬行动物有哪些 高胜美|||怕爬行动物
***response***
黄志玮有什么喜欢的卡通？ 黄志玮|||喜欢的卡通
*

In [37]:
sample

{'id': 'TE000267',
 'question': '迪迦·奥特曼的结束曲是哪一首？',
 'attribute': [['40014094', '迪迦·奥特曼 ||| 别名 ||| 迪迦·奥特曼'],
  ['41041965', '迪迦奥特曼 ||| 别名 ||| 迪迦奥特曼'],
  ['39974452', '《迪迦奥特曼》 ||| 别名 ||| 《迪迦奥特曼》'],
  ['39653423', '《迪迦·奥特曼》 ||| 别名 ||| 《迪迦·奥特曼》'],
  ['19569354', '邪恶迪迦(《迪迦奥特曼》TV版中的邪恶奥特曼) ||| 宿敌 ||| 迪迦奥特曼'],
  ['41041966', '迪迦奥特曼 ||| 中文名 ||| 迪迦奥特曼'],
  ['40014106', '迪迦·奥特曼 ||| 中文名 ||| 迪迦·奥特曼'],
  ['40014113', '迪迦·奥特曼 ||| 迪迦·奥特曼开场画面 ||| 迪迦·奥特曼开场画面'],
  ['7261991', '迪迦奥特曼7 ||| 别名 ||| 迪迦奥特曼7'],
  ['11154962', '迪迦奥特曼12 ||| 别名 ||| 迪迦奥特曼12'],
  ['12707084', '迪迦奥特曼2 ||| 书名 ||| 迪迦奥特曼2'],
  ['17343825', '迪迦奥特曼4 ||| 书名 ||| 迪迦奥特曼4'],
  ['31843426', '迪迦奥特曼Z ||| 别名 ||| 迪迦奥特曼Z'],
  ['40041242', '《迪迦奥特曼Z》 ||| 别名 ||| 《迪迦奥特曼Z》'],
  ['12079633', '光迪迦奥特曼 ||| 别名 ||| 光迪迦奥特曼'],
  ['12705792', '迪迦奥特曼10 ||| 书名 ||| 迪迦奥特曼10'],
  ['14514088', '迪迦奥特曼11 ||| 别名 ||| 迪迦奥特曼11'],
  ['17739940', '迪迦奥特曼1 ||| 别名 ||| 迪迦奥特曼1'],
  ['23695839', '迪迦奥特曼9 ||| 别名 ||| 迪迦奥特曼9'],
  ['36973525', '迪迦奥特曼13 ||| 别名 ||| 迪迦奥特曼13']]}

In [40]:
submission = open("../../result/test_tuple_extract_v1.json", "w", encoding="utf8")

submission.write(json.dumps(data, ensure_ascii=False))
# print(len(submission),len(data))
print(data[0])

{'id': 'TE000138', 'question': '恩金·巴伊塔尔作为土耳其运动员，在球队中担负的职责是什么', 'attribute': ['恩金·巴伊塔尔|||所属运动队']}


# final_result

In [103]:
submission = open("../../result/es_llm_top20_v1.json", "w", encoding="utf8")

submission.write(json.dumps(data, ensure_ascii=False))
# print(len(submission),len(data))
print(data[0])

{'id': 'TE000138', 'question': '恩金·巴伊塔尔作为土耳其运动员，在球队中担负的职责是什么', 'attribute': ['没有找到该问题对应的知识']}


In [15]:
import json
from tqdm import tqdm_notebook
train_data = [json.loads(k) for k in open("../../data/dev_sft.json", encoding="utf8")]

In [ ]:
sums,count=0,0
true,false=[],[]
for unit in tqdm_notebook(train_data):
    if unit["task_type"]=="table_extract":
        prompt = "已知下面表格信息：" + "{}，\n问：{}\n答：".format(";".join([k for k in unit["instruction"]]), unit["question"])

        response, history=model.chat(tokenizer,prompt,max_length=2048)
        response=";".join(response.split("；"))
        sums+=1
        if response==unit["answer"]:
            count+=1
            true.append((response,unit["answer"]))
        else:
            false.append((response,unit["answer"]))

/tmp/ipykernel_3763/2164909142.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for unit in tqdm_notebook(train_data):


  0%|          | 0/470 [00:00<?, ?it/s]

In [31]:
prompt

'已知下面表格信息：是;说;的;是;是;心;爱;的;说;，;究;谁;是;是;究;，;是;心;是;谁，\n问：爱德华·费根鲍姆是一位知名的科学家，他在计算机科学领域有着广泛的研究和丰富的成就。但我对他的具体信息了解甚少，你可以告诉我他的出生日期，研究领域，他在哪里工作，他的母校是哪里，他的导师是谁，他的主要成就或者获奖情况吗？\n答：'

In [44]:
count,sums

(4, 4)

In [40]:
unit["answer"]

'古宇峰 ||| 英文名 ||| GuYufeng'